## Загрузим нужные библиотеки

In [67]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import numpy as np
from sklearn.metrics import recall_score, precision_score, mean_absolute_error, mean_squared_error, accuracy_score

import plotly.express as px

%matplotlib inline

В этом примере мы попробуем предсказать итоговую оценку участинка только на основе **его действий**

In [68]:
df_train_source = pd.read_csv("/content/train_dataset_train.csv")
df_test_source = pd.read_csv('/content/sample_solution.csv')
df_decision = pd.read_csv("/content/user_decision.csv")

In [69]:
df_user = pd.read_csv("/content/user.csv")

In [70]:
df_team = pd.read_csv("/content/team_point.csv")

### Подготовка данных для обучения, преобразование всех действий для игрока в вектора, где в каждый индекс записано количество действий совершенных игроком в ходе игры

In [71]:
new_x_train = []
new_y_train_analytical_thinking = []
new_y_train_systemic_thinking = []
new_y_train_adaptability = []
new_y_train_focus = []

dict_new_x_train = {}
dict_new_y_train_analytical_thinking = {}
dict_new_y_train_systemic_thinking = {}
dict_new_y_train_adaptability = {}
dict_new_y_train_focus = {}

new_x_test = []
new_y_test_analytical_thinking = []
new_y_test_systemic_thinking = []
new_y_test_adaptability = []
new_y_test_focus = []

dict_new_x_test = {}
dict_new_y_test_analytical_thinking = {}
dict_new_y_test_systemic_thinking = {}
dict_new_y_test_adaptability = {}
dict_new_y_test_focus = {}

In [73]:
use_team = False

In [74]:
use_period = False

In [75]:
# проходим по всем коммандам
for idx, row in df_user.iterrows():
  user_id = row['user_id']
  
  # получаем всех игроков в команде
  users_in_team = df_user[df_user['team_id'] == row['team_id']]['user_id'].values
  # вектор решений всей команды
  decision_team = [0] * 438
  
  if use_team:
    # проходим по игрокам в команде
    for user_in_team_id in users_in_team:
      # получаем действия игрога команды
      user_decisions = df_decision[df_decision['user_id'] == user_in_team_id]['decision_id'].values

      count_val = 2 if user_id == user_in_team_id else 1
      for user_decision in user_decisions:
        # каждое действие это индекс прибавляем счетчик действий
        decision_team[user_decision] += count_val
  elif use_period:
    user_decisions = df_decision[df_decision['user_id'] == user_id]
    for idx_user, row_decision in user_decisions.iterrows():
      decision_id = row_decision['decision_id']
      period = row_decision['period']
      # каждое действие это индекс прибавляем счетчик действий
      decision_team[decision_id] += 1 * period
  else:
    user_decisions = df_decision[df_decision['user_id'] == user_id]['decision_id'].values
    for user_decision in user_decisions:
      # каждое действие это индекс прибавляем счетчик действий
      decision_team[user_decision] += 1

  

  # получаем оценки игрока команды
  rates = df_train_source[df_train_source['id'] == user_id]
  analytical_thinking = rates['Analytical thinking'].values
  systemic_thinking = rates['Systemic thinking'].values
  adaptability = rates['Adaptability'].values
  focus = rates['Focus'].values

  rates_test = df_test_source[df_test_source['id'] == user_id]
  analytical_thinking_test = rates_test['Analytical thinking'].values
  systemic_thinking_test = rates_test['Systemic thinking'].values
  adaptability_test = rates_test['Adaptability'].values
  focus_test = rates_test['Focus'].values

  if len(analytical_thinking) > 0:
    dict_new_x_train[user_id] = decision_team
    dict_new_y_train_analytical_thinking[user_id] = analytical_thinking[0]
    dict_new_y_train_systemic_thinking[user_id] = systemic_thinking[0]
    dict_new_y_train_adaptability[user_id] = adaptability[0]
    dict_new_y_train_focus[user_id] = focus[0]

  if len(analytical_thinking_test) > 0:
    dict_new_x_test[user_id] = decision_team
    dict_new_y_test_analytical_thinking[user_id] = analytical_thinking_test[0]
    dict_new_y_test_systemic_thinking[user_id] = systemic_thinking_test[0]
    dict_new_y_test_adaptability[user_id] = adaptability_test[0]
    dict_new_y_test_focus[user_id] = focus_test[0]


In [76]:
for idx, row in df_train_source.iterrows():
  user_id = row['id']
  new_x_train.append(dict_new_x_train[user_id])
  new_y_train_analytical_thinking.append(dict_new_y_train_analytical_thinking[user_id])
  new_y_train_systemic_thinking.append(dict_new_y_train_systemic_thinking[user_id])
  new_y_train_adaptability.append(dict_new_y_train_adaptability[user_id])
  new_y_train_focus.append(dict_new_y_train_focus[user_id])

In [77]:
for idx, row in df_test_source.iterrows():
  user_id = row['id']
  new_x_test.append(dict_new_x_test[user_id])
  new_y_test_analytical_thinking.append(dict_new_y_test_analytical_thinking[user_id])
  new_y_test_systemic_thinking.append(dict_new_y_test_systemic_thinking[user_id])
  new_y_test_adaptability.append(dict_new_y_test_adaptability[user_id])
  new_y_test_focus.append(dict_new_y_test_focus[user_id])

In [78]:
new_x_train = np.array(new_x_train)
new_y_train_analytical_thinking = np.array(new_y_train_analytical_thinking)
new_y_train_systemic_thinking = np.array(new_y_train_systemic_thinking)
new_y_train_adaptability = np.array(new_y_train_adaptability)
new_y_train_focus = np.array(new_y_train_focus)
new_x_test = np.array(new_x_test)
new_y_test_analytical_thinking = np.array(new_y_test_analytical_thinking)
new_y_test_systemic_thinking = np.array(new_y_test_systemic_thinking)
new_y_test_adaptability = np.array(new_y_test_adaptability)
new_y_test_focus = np.array(new_y_test_focus)

### Удаление действий которых никогда не было за все игры

In [79]:
df_temp1 = pd.DataFrame(new_x_train, columns = range(438))

In [80]:
df_temp3 = pd.DataFrame(new_x_test, columns = range(438))

In [81]:
zero_indexes = [0,
 2,
 19,
 20,
 22,
 23,
 24,
 37,
 38,
 39,
 40,
 41,
 43,
 45,
 157,
 158,
 162,
 163,
 164,
 169,
 198,
 208,
 215,
 217,
 218,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 234,
 236,
 237,
 238,
 240,
 241,
 242,
 243,
 257,
 260,
 262,
 263,
 264,
 279,
 280,
 281,
 282,
 283,
 299,
 305,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 402,
 403,
 404,
 416,
 417,
 431]

In [82]:
df_temp1 = df_temp1.drop(df_temp1.columns[zero_indexes],axis=1)

In [83]:
df_temp3 = df_temp3.drop(df_temp3.columns[zero_indexes],axis=1)

In [84]:
new_x_train = df_temp1.to_numpy()

In [85]:
new_x_test = df_temp3.to_numpy()

## Обучение модели

In [86]:
from sklearn.tree import DecisionTreeClassifier

In [87]:
from sklearn.experimental import enable_hist_gradient_boosting

In [88]:
from sklearn.svm import LinearSVC

In [89]:
from sklearn.ensemble import AdaBoostClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier

In [90]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [91]:
from sklearn.linear_model import SGDClassifier, PassiveAggressiveClassifier

In [92]:
from sklearn.neural_network import MLPClassifier

## systemic thinking

In [93]:
X_train, X_validation, Y_train, Y_validation = train_test_split(new_x_train, new_y_train_systemic_thinking, train_size=0.90, random_state=45)

In [94]:
st_hgc1 = HistGradientBoostingClassifier(early_stopping=False,
                                 l2_regularization=1.0179394834253815e-10,
                                 learning_rate=0.263564230193214, max_iter=512,
                                 min_samples_leaf=35, n_iter_no_change=0,
                                 random_state=1, validation_fraction=None,
                                 warm_start=True)
st_hgc1.fit(X_train, Y_train)
hgc1_pred = st_hgc1.predict(X_validation)
print('Accuracy: ', accuracy_score(Y_validation, hgc1_pred))
print('MAE: ', mean_absolute_error(Y_validation, hgc1_pred))
print('Recall: ', recall_score(Y_validation, hgc1_pred, average='macro', zero_division=True))

Accuracy:  0.8795180722891566
MAE:  0.12650602409638553
Recall:  0.675672514619883


In [95]:
st_hgc1 = HistGradientBoostingClassifier(early_stopping=False,
                                 l2_regularization=1.0179394834253815e-10,
                                 learning_rate=0.263564230193214, max_iter=512,
                                 min_samples_leaf=35, n_iter_no_change=0,
                                 random_state=1, validation_fraction=None,
                                 warm_start=True)
st_hgc1.fit(new_x_train, new_y_train_systemic_thinking)
hgc1_pred = st_hgc1.predict(X_validation)
print(accuracy_score(Y_validation, hgc1_pred))
print(mean_absolute_error(Y_validation, hgc1_pred))
print(recall_score(Y_validation, hgc1_pred, average='macro', zero_division=True))

1.0
0.0
1.0


### test predict

In [96]:
test_systemic_thinking = st_hgc1.predict(new_x_test)

In [97]:
test_systemic_thinking

array([3., 4., 4., 4., 4., 4., 6., 4., 4., 3., 4., 4., 4., 4., 5., 5., 4.,
       5., 4., 4., 4., 4., 4., 4., 1., 4., 5., 4., 4., 5., 4., 6., 4., 4.,
       4., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 5., 3., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 5., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 5., 4., 4., 4., 4., 5., 3., 4., 4., 4., 4., 4., 4., 5., 4.,
       4., 4., 4., 5., 3., 4., 4., 5., 3., 4., 4., 3., 4., 4., 5., 4., 4.,
       4., 3., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 4.,
       4., 5., 5., 5., 3., 4., 4., 4., 4., 4., 6., 4., 4., 5., 4., 4., 4.,
       4., 4., 4., 3., 4., 3., 5., 4., 4., 4., 4., 5., 5., 4., 4., 4., 5.,
       5., 4., 4., 3., 5., 5., 4., 4., 4., 1., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 3., 4., 4., 4., 4., 5., 4., 3., 4., 4., 5., 4., 4.,
       4., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 5., 3., 5., 4., 4.,
       6., 5., 4., 5., 4., 4., 5., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 3.

## Analytical thinking

In [98]:
X_train, X_validation, Y_train, Y_validation = train_test_split(new_x_train, new_y_train_analytical_thinking, train_size=0.90, random_state=45)

In [99]:
at_hgc1 = RandomForestClassifier(max_features=32, n_estimators=512, n_jobs=1,
                         random_state=1, warm_start=True)
at_hgc1.fit(X_train, Y_train)
hgc1_pred = at_hgc1.predict(X_validation)
print('Accuracy: ', accuracy_score(Y_validation, hgc1_pred))
print('MAE: ', mean_absolute_error(Y_validation, hgc1_pred))
print('Recall: ', recall_score(Y_validation, hgc1_pred, average='macro', zero_division=True))

Accuracy:  0.8855421686746988
MAE:  0.1144578313253012
Recall:  0.8419825376959295


In [100]:
at_hgc1 = RandomForestClassifier(max_features=32, n_estimators=512, n_jobs=1,
                         random_state=1, warm_start=True)
at_hgc1.fit(new_x_train, new_y_train_analytical_thinking)
hgc1_pred = at_hgc1.predict(X_validation)
print(accuracy_score(Y_validation, hgc1_pred))
print(mean_absolute_error(Y_validation, hgc1_pred))
print(recall_score(Y_validation, hgc1_pred, average='macro', zero_division=True))

1.0
0.0
1.0


### test predict

In [101]:
test_analytical_thinking = at_hgc1.predict(new_x_test)

In [102]:
test_analytical_thinking

array([4., 4., 3., 4., 5., 5., 6., 5., 5., 4., 4., 5., 4., 4., 6., 5., 4.,
       5., 5., 4., 4., 5., 4., 4., 3., 4., 5., 4., 4., 5., 4., 6., 4., 5.,
       4., 4., 3., 4., 5., 4., 5., 4., 5., 4., 5., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 5., 4., 4., 5., 4., 5., 5., 4., 4., 4., 4.,
       4., 5., 4., 4., 4., 4., 4., 5., 4., 4., 4., 5., 4., 4., 4., 4., 5.,
       4., 5., 4., 5., 4., 4., 5., 4., 4., 4., 4., 3., 5., 4., 5., 5., 5.,
       5., 4., 4., 5., 4., 4., 4., 5., 4., 5., 5., 4., 4., 5., 5., 5., 4.,
       4., 5., 5., 5., 4., 4., 5., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4.,
       4., 5., 4., 4., 4., 4., 6., 4., 4., 4., 4., 4., 4., 5., 5., 3., 4.,
       6., 4., 4., 3., 4., 5., 4., 4., 4., 3., 5., 5., 4., 4., 5., 4., 4.,
       4., 5., 4., 5., 4., 4., 4., 4., 5., 4., 5., 4., 5., 4., 4., 5., 5.,
       5., 5., 5., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 5., 5., 4.,
       5., 5., 4., 4., 4., 4., 4., 4., 5., 5., 5., 4., 4., 4., 4., 4., 4.,
       4., 5., 4., 5., 4.

## Adaptability

In [103]:
X_train, X_validation, Y_train, Y_validation = train_test_split(new_x_train, new_y_train_adaptability, train_size=0.90, random_state=45)

In [104]:
ad_hgc1 = RandomForestClassifier(max_features=8, n_estimators=512, n_jobs=1,
                         random_state=1, warm_start=True)
ad_hgc1.fit(X_train, Y_train)
hgc1_pred = ad_hgc1.predict(X_validation)
print('Accuracy: ', accuracy_score(Y_validation, hgc1_pred))
print('MAE: ', mean_absolute_error(Y_validation, hgc1_pred))
print('Recall: ', recall_score(Y_validation, hgc1_pred, average='macro', zero_division=True))

Accuracy:  0.891566265060241
MAE:  0.1144578313253012
Recall:  0.7381538461538462


In [105]:
ad_hgc1 = RandomForestClassifier(max_features=8, n_estimators=512, n_jobs=1,
                         random_state=1, warm_start=True)
ad_hgc1.fit(new_x_train, new_y_train_adaptability)
hgc1_pred = ad_hgc1.predict(X_validation)
print(accuracy_score(Y_validation, hgc1_pred))
print(mean_absolute_error(Y_validation, hgc1_pred))
print(recall_score(Y_validation, hgc1_pred, average='macro', zero_division=True))

1.0
0.0
1.0


### test predict

In [106]:
test_adaptability = ad_hgc1.predict(new_x_test)

In [107]:
test_adaptability

array([5., 5., 4., 4., 5., 4., 5., 5., 5., 4., 4., 5., 4., 5., 5., 5., 4.,
       5., 5., 4., 5., 5., 5., 5., 1., 5., 5., 4., 4., 4., 4., 5., 4., 5.,
       5., 4., 4., 4., 5., 5., 5., 4., 5., 5., 5., 5., 5., 4., 5., 4., 5.,
       5., 4., 4., 4., 5., 5., 5., 4., 4., 5., 5., 5., 5., 4., 5., 5., 5.,
       5., 5., 5., 4., 5., 4., 4., 5., 3., 4., 5., 5., 5., 4., 5., 5., 5.,
       4., 5., 4., 5., 4., 4., 4., 4., 4., 5., 5., 4., 4., 5., 5., 4., 4.,
       5., 4., 4., 4., 5., 5., 4., 5., 4., 5., 5., 5., 4., 5., 5., 5., 5.,
       4., 4., 5., 5., 4., 4., 5., 5., 5., 4., 5., 4., 4., 5., 5., 5., 5.,
       5., 5., 4., 4., 4., 4., 5., 4., 5., 4., 4., 5., 5., 5., 5., 4., 5.,
       5., 5., 5., 4., 4., 5., 4., 4., 5., 1., 5., 5., 4., 4., 4., 5., 5.,
       4., 5., 4., 4., 4., 4., 5., 4., 5., 4., 5., 5., 5., 4., 5., 5., 5.,
       4., 5., 5., 5., 5., 5., 5., 4., 5., 4., 5., 5., 5., 4., 5., 4., 5.,
       5., 5., 4., 5., 4., 4., 4., 4., 5., 4., 5., 5., 4., 5., 5., 5., 5.,
       4., 4., 4., 5., 5.

## Focus

In [108]:
X_train, X_validation, Y_train, Y_validation = train_test_split(new_x_train, new_y_train_focus, train_size=0.90, random_state=45)

In [109]:
f_hgc1 = RandomForestClassifier(max_features=4, n_estimators=512, n_jobs=1,
                         random_state=1, warm_start=True)
f_hgc1.fit(X_train, Y_train)
hgc1_pred = f_hgc1.predict(X_validation)
print('Accuracy: ', accuracy_score(Y_validation, hgc1_pred))
print('MAE: ', mean_absolute_error(Y_validation, hgc1_pred))
print('Recall: ', recall_score(Y_validation, hgc1_pred, average='macro', zero_division=True))

Accuracy:  0.8855421686746988
MAE:  0.1144578313253012
Recall:  0.8542517006802721


In [110]:
f_hgc1 = RandomForestClassifier(max_features=4, n_estimators=512, n_jobs=1,
                         random_state=1, warm_start=True)
f_hgc1.fit(new_x_train, new_y_train_focus)
hgc1_pred = f_hgc1.predict(X_validation)
print(accuracy_score(Y_validation, hgc1_pred))
print(mean_absolute_error(Y_validation, hgc1_pred))
print(recall_score(Y_validation, hgc1_pred, average='macro', zero_division=True))

1.0
0.0
1.0


### test predict

In [111]:
test_focus = f_hgc1.predict(new_x_test)

In [112]:
test_focus

array([4., 5., 4., 4., 4., 4., 5., 5., 4., 3., 4., 4., 4., 4., 5., 5., 4.,
       4., 5., 4., 5., 4., 4., 4., 3., 5., 5., 5., 4., 4., 4., 5., 3., 5.,
       4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 5., 5., 5., 4., 4., 4., 4.,
       4., 5., 4., 4., 5., 4., 5., 4., 3., 5., 5., 4., 5., 4., 4., 5., 4.,
       4., 5., 4., 4., 4., 4., 5., 5., 3., 5., 4., 5., 4., 4., 4., 5., 4.,
       4., 5., 3., 5., 4., 4., 4., 5., 4., 4., 5., 4., 4., 4., 5., 4., 4.,
       5., 3., 4., 4., 4., 4., 4., 5., 4., 5., 4., 4., 4., 5., 5., 4., 5.,
       4., 5., 4., 5., 4., 4., 4., 5., 4., 4., 5., 4., 4., 5., 5., 5., 5.,
       4., 4., 5., 4., 4., 3., 5., 5., 4., 3., 5., 4., 4., 4., 5., 4., 4.,
       5., 4., 4., 4., 4., 6., 4., 5., 4., 3., 5., 5., 4., 3., 4., 4., 5.,
       4., 4., 4., 4., 3., 4., 4., 4., 5., 4., 4., 4., 4., 4., 5., 5., 4.,
       5., 4., 4., 4., 4., 5., 5., 4., 5., 4., 4., 5., 5., 4., 5., 4., 4.,
       5., 5., 5., 4., 4., 4., 4., 4., 4., 5., 5., 4., 3., 4., 5., 4., 4.,
       5., 4., 4., 5., 4.

### Заполение результатов предсказания и сохранение в csv

In [113]:
df_test_source['Analytical thinking'] = test_analytical_thinking
df_test_source['Systemic thinking'] = test_systemic_thinking
df_test_source['Adaptability'] = test_adaptability
df_test_source['Focus'] = test_focus

In [114]:
df_test_source

,id,Analytical thinking,Systemic thinking,Adaptability,Focus
0,10199,4.0,3.0,5.0,4.0
1,10539,4.0,4.0,5.0,5.0
2,10174,3.0,4.0,4.0,4.0
3,10465,4.0,4.0,4.0,4.0
4,10066,5.0,4.0,5.0,4.0
...,...,...,...,...,...
288,10433,5.0,5.0,5.0,5.0
289,10893,4.0,4.0,5.0,4.0
290,10909,4.0,4.0,5.0,4.0
291,10889,3.0,3.0,3.0,3.0


In [115]:
df_test_source.to_csv('simple.csv', index=False)